In [ ]:
import time

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from scipy.stats import kurtosis, skew
from sklearn.metrics import roc_curve, auc
from itertools import cycle

from scipy.fft import fft
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import seaborn as sns

from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA

from sklearn.preprocessing import label_binarize

from sklearn.preprocessing import MinMaxScaler

In [ ]:
#File names

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# --------------------------------------------------------

# Input Dataset I

In [ ]:


data0D = pd.read_csv('/kaggle/input/vibration-dataset-1/0D.csv')
data1D = pd.read_csv('/kaggle/input/vibration-dataset-1/1D.csv')
data2D = pd.read_csv('/kaggle/input/vibration-dataset-1/2D.csv')
data3D = pd.read_csv('/kaggle/input/vibration-dataset-1/3D.csv')
data4D = pd.read_csv('/kaggle/input/vibration-dataset-1/4D.csv')

data0E = pd.read_csv('/kaggle/input/vibration-dataset-1/0E.csv')
data1E = pd.read_csv('/kaggle/input/vibration-dataset-1/1E.csv')
data2E = pd.read_csv('/kaggle/input/vibration-dataset-1/2E.csv')
data3E = pd.read_csv('/kaggle/input/vibration-dataset-1/3E.csv')
data4E = pd.read_csv('/kaggle/input/vibration-dataset-1/4E.csv')



print('DATASET LOADED')

In [ ]:
# The signal was initialized from 20 seconds onward to eliminate the initial transient phase.

fs = 4096
initial_time = 20 * fs  


# Reset index
data0D = data0D.iloc[initial_time:].reset_index(drop=True)
data1D = data1D.iloc[initial_time:].reset_index(drop=True)
data2D = data2D.iloc[initial_time:].reset_index(drop=True)
data3D = data3D.iloc[initial_time:].reset_index(drop=True)
data4D = data4D.iloc[initial_time:].reset_index(drop=True)


data0E = data0E.iloc[initial_time:].reset_index(drop=True)
data1E = data1E.iloc[initial_time:].reset_index(drop=True)
data2E = data2E.iloc[initial_time:].reset_index(drop=True)
data3E = data3E.iloc[initial_time:].reset_index(drop=True)
data4E = data4E.iloc[initial_time:].reset_index(drop=True)


print('Done')

In [ ]:
# One-second window

window_time = 1
window = fs * window_time  

### Divide the raw signal in samples of 1 sec and labeling each sample

In [ ]:
# Extracts signal segments of the specified window size.

def get_features(data, label):
    n = int(np.floor(len(data)/window))
    data = data[:int(n)*window]
    X = data.values.reshape((n, window))
    y = np.ones(n)*labels[label]
    return X,y

In [ ]:
labels = {'no_unbalance':0, 'unbalance_1':1, 'unbalance_2':2,'unbalance_3':3, 'unbalance_4':4}
sensor = 'Vibration_2'


X0D, y0D = get_features(data0D[sensor], "no_unbalance")
X1D, y1D = get_features(data1D[sensor], "unbalance_1")
X2D, y2D = get_features(data2D[sensor], "unbalance_2")
X3D, y3D = get_features(data3D[sensor], "unbalance_3")
X4D, y4D = get_features(data4D[sensor], "unbalance_4")


X0E, y0E = get_features(data0E[sensor], "no_unbalance")
X1E, y1E = get_features(data1E[sensor], "unbalance_1")
X2E, y2E = get_features(data2E[sensor], "unbalance_2")
X3E, y3E = get_features(data3E[sensor], "unbalance_3")
X4E, y4E = get_features(data4E[sensor], "unbalance_4")


X=np.concatenate([X0D, X1D, X2D, X3D, X4D, X0E, X1E, X2E, X3E, X4E])
Y=np.concatenate([y0D, y1D, y2D, y3D, y4D, y0E, y1E, y2E, y3E, y4E])


print(X.shape, Y.shape)

In [ ]:
X4D_1, y4D_1 = get_features(data4D['Vibration_1'], "unbalance_4")
X4D_2, y4D_2 = get_features(data4D['Vibration_2'], "unbalance_4")
X4D_3, y4D_3 = get_features(data4D['Vibration_3'], "unbalance_4")

t = np.arange(window) / fs

indice = 5


p2p = lambda x: float(np.max(x) - np.min(x))

rms = lambda x: float(np.sqrt(np.mean(x**2)))


p2p_vals = {
    'Vibration_1': p2p(X4D_1[indice]),
    'Vibration_2': p2p(X4D_2[indice]),
    'Vibration_3': p2p(X4D_3[indice]),
}

rms_vals = {
    'Vibration_1': rms(X4D_1[indice]),
    'Vibration_2': rms(X4D_2[indice]),
    'Vibration_3': rms(X4D_3[indice]),
}

v1 = X4D_1[indice] - np.mean(X4D_1[indice])
v2 = X4D_2[indice] - np.mean(X4D_2[indice])
v3 = X4D_3[indice] - np.mean(X4D_3[indice])


plt.figure(figsize=(12,5))
plt.plot(t, v1, label=f'Vibration_1 (p2p={p2p_vals["Vibration_1"]:.3f},RMS={rms_vals["Vibration_1"]:.3f}))')
plt.plot(t, v2, label=f'Vibration_2 (p2p={p2p_vals["Vibration_2"]:.3f},RMS={rms_vals["Vibration_2"]:.3f}))')
plt.plot(t, v3, label=f'Vibration_3 (p2p={p2p_vals["Vibration_3"]:.3f},RMS={rms_vals["Vibration_3"]:.3f}))')
plt.xlim(0, 1)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Dataset I - Comparison of the three-axis vibration signals for a single sample at the 4th degree of unbalance')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()



print('Pico-a-pico por eixo:', p2p_vals)
print('Eixo com maior amplitude:', max(p2p_vals, key=p2p_vals.get))

# --------------------------------------------------------

# Input Dataset II

In [ ]:

for i in range(1, 1001):
    globals()[f"data_normal_{i}"] = pd.read_csv(f'/kaggle/input/vbl-va001/normal/normal_{i}.csv', header=None)

for i in range(1, 501):
    globals()[f"data_unbalance_i_{i}"] = pd.read_csv(f'/kaggle/input/vbl-va001/unbalance_6/unbalance_i_{i}.csv', header=None)

for i in range(1, 501):
    globals()[f"data_unbalance_ii_{i}"] = pd.read_csv(f'/kaggle/input/vbl-va001/unbalance_27/unbalance_ii_{i}.csv', header=None)



for i in range(1, 1001):
    globals()[f"data_misalignment_{i}"] = pd.read_csv(f'/kaggle/input/vbl-va001/misalignment/misalignment_{i}.csv', header=None)
    

for i in range(1, 1001):
    globals()[f"data_bearing_{i}"] = pd.read_csv(f'/kaggle/input/vbl-va001/bearing/bearing_{i}.csv', header=None)



print('DATASET LOADED')

In [ ]:
# Visualize which axis exhibits the highest amplitude.


df = globals()["data_unbalance_ii_100"]

time = df.iloc[:, 0]
axis_x = df.iloc[:, 1]
axis_y = df.iloc[:, 2]
axis_z = df.iloc[:, 3]

plt.figure(figsize=(12, 6))
plt.plot(time, axis_x, label='X axis')
plt.plot(time, axis_y, label='Y axis')
plt.plot(time, axis_z, label='Z axis')
plt.title('Vibration in 3 axis - Unbalance II')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
fs = 20000

window_time = 1
window = fs * window_time  


### Divide the raw signal in samples of 1 sec and labeling each sample

In [ ]:
# Extracts signal segments of the specified window size.
def get_features(data, label):
    n = int(np.floor(len(data)/window))
    data = data[:int(n)*window]
    X = data.values.reshape((n, window))
    y = np.ones(n)*labels[label]
    return X,y

In [ ]:

labels = {'normal':0, 'unbalance_1':1, 'unbalance_2':2,'misaligment':3, 'bearing_fault':4}
axis = 2


X_list = []
Y_list = []


for i in range(1, 1001):
    globals()[f'X_normal_{i}'], globals()[f'y_normal_{i}'] = get_features(globals()[f'data_normal_{i}'][axis], "normal")
    X_list.append(globals()[f"X_normal_{i}"])
    Y_list.append(globals()[f"y_normal_{i}"])


for i in range(1, 501):
    globals()[f'X_unbalance_i_{i}'], globals()[f'y_unbalance_i_{i}'] = get_features(globals()[f'data_unbalance_i_{i}'][axis], "unbalance_1")
    X_list.append(globals()[f"X_unbalance_i_{i}"])
    Y_list.append(globals()[f"y_unbalance_i_{i}"])
    
for i in range(1, 501):
    globals()[f'X_unbalance_ii_{i}'], globals()[f'y_unbalance_ii_{i}'] = get_features(globals()[f'data_unbalance_ii_{i}'][axis], "unbalance_2")
    X_list.append(globals()[f"X_unbalance_ii_{i}"])
    Y_list.append(globals()[f"y_unbalance_ii_{i}"])




for i in range(1, 1001):
    globals()[f'X_misaligment_{i}'], globals()[f'y_misaligment_{i}'] = get_features(globals()[f'data_misalignment_{i}'][axis], "misaligment")
    X_list.append(globals()[f"X_misaligment_{i}"])
    Y_list.append(globals()[f"y_misaligment_{i}"])

for i in range(1, 1001):
    globals()[f'X_bearing_{i}'], globals()[f'y_bearing_{i}'] = get_features(globals()[f'data_bearing_{i}'][axis], "bearing_fault")
    X_list.append(globals()[f"X_bearing_{i}"])
    Y_list.append(globals()[f"y_bearing_{i}"])



X=np.concatenate(X_list)
Y=np.concatenate(Y_list)


print(X.shape, Y.shape)

print('Done')

# --------------------------------------------------------

# Input Dataset III

In [ ]:

for i in range(1, 50):
    globals()[f"data_normal_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/normal/normal_{i}.csv', header=None, sep = '[;,]', engine = 'python')

for i in range(1, 49):
    globals()[f"data_unbalance_i_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/unbalance_i/unbalance_6_{i}.csv', header=None, sep = '[;,]', engine = 'python')

for i in range(1, 49):
    globals()[f"data_unbalance_ii_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/unbalance_ii/unbalance_20_{i}.csv', header=None, sep = '[;,]', engine = 'python')

for i in range(1, 49):
    globals()[f"data_unbalance_iii_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/unbalance_iii/unbalance_35_{i}.csv', header=None, sep = '[;,]', engine = 'python')


for i in range(1, 50):
    globals()[f"data_misalignment_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/misalignment/misalignment_{i}.csv', header=None,sep = '[;,]', engine = 'python')


for i in range(1, 40):
    globals()[f"data_unbalance_misaligment_{i}"] = pd.read_csv(f'/kaggle/input/comfaulda/COMFAULDA_v2/unbalance_misalignment/unbalance_misalignment_{i}.csv', header=None, sep = '[;,]', engine = 'python')


print('DATASET LOADED')

In [ ]:
# Visualize which axis exhibits the highest amplitude.

df = globals()["data_unbalance_ii_42"]

axis_time = df.iloc[:, 0]
axis_x = df.iloc[:, 5]
axis_y = df.iloc[:, 7]
axis_z = df.iloc[:, 6]

plt.figure(figsize=(12, 6))
plt.plot(time, axis_x, label='X axis')
plt.plot(time, axis_y, label='Y axis')
plt.plot(time, axis_z, label='Z axis')
plt.title('Vibration in 3 axis - Unbalance II')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
fs = 50000

window_time = 1 
window = fs * window_time  

In [ ]:
# Extracts signal segments of the specified window size.

def get_features(data, label):
    n = int(np.floor(len(data)/window))
    data = data[:int(n)*window]
    X = data.values.reshape((n, window))
    y = np.ones(n)*labels[label]
    return X,y

In [ ]:

labels = {'normal':0, 'unbalance_1':1, 'unbalance_2':2, 'unbalance_3':3, 'misaligment':4, 'unbalance_misaligment':5}
axis = 6


X_list = []
Y_list = []


for i in range(1, 50):
    globals()[f'X_normal_{i}'], globals()[f'y_normal_{i}'] = get_features(globals()[f'data_normal_{i}'][axis], "normal")
    X_list.append(globals()[f"X_normal_{i}"])
    Y_list.append(globals()[f"y_normal_{i}"])

for i in range(1, 49):
    globals()[f'X_unbalance_i_{i}'], globals()[f'y_unbalance_i_{i}'] = get_features(globals()[f'data_unbalance_i_{i}'][axis], "unbalance_1")
    X_list.append(globals()[f"X_unbalance_i_{i}"])
    Y_list.append(globals()[f"y_unbalance_i_{i}"])
    
for i in range(1, 49):
    globals()[f'X_unbalance_ii_{i}'], globals()[f'y_unbalance_ii_{i}'] = get_features(globals()[f'data_unbalance_ii_{i}'][axis], "unbalance_2")
    X_list.append(globals()[f"X_unbalance_ii_{i}"])
    Y_list.append(globals()[f"y_unbalance_ii_{i}"])

for i in range(1, 49):
    globals()[f'X_unbalance_iii_{i}'], globals()[f'y_unbalance_iii_{i}'] = get_features(globals()[f'data_unbalance_iii_{i}'][axis], "unbalance_3")
    X_list.append(globals()[f"X_unbalance_iii_{i}"])
    Y_list.append(globals()[f"y_unbalance_iii_{i}"])


for i in range(1, 50):
    globals()[f'X_misalignment_{i}'], globals()[f'y_misalignment_{i}'] = get_features(globals()[f'data_misalignment_{i}'][axis], "misaligment")
    X_list.append(globals()[f"X_misalignment_{i}"])
    Y_list.append(globals()[f"y_misalignment_{i}"])

for i in range(1, 40):
    globals()[f'X_unbalance_misaligment_{i}'], globals()[f'y_unbalance_misaligment_{i}'] = get_features(globals()[f'data_unbalance_misaligment_{i}'][axis], "unbalance_misaligment")
    X_list.append(globals()[f"X_unbalance_misaligment_{i}"])
    Y_list.append(globals()[f"y_unbalance_misaligment_{i}"])



X=np.concatenate(X_list)
Y=np.concatenate(Y_list)


print(X.shape, Y.shape)

print('Done')


# --------------------------------------------------------

## Train, validation and test dataset split

### 80% Train and 20% Test

In [ ]:
X, Y = shuffle(X, Y, random_state=42)

In [ ]:
# Extracts signal segments of the specified window size.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

## 01 - Random Forest with statistical features extraction

### Feature extraction

In [ ]:

def extract_features(signal):
    features = []
    features.append(np.max(signal))     # maximum
    features.append(np.min(signal))     # minimum
    features.append(np.max(np.abs(signal))) #Peak
    features.append(np.ptp(signal))         # #Peak-to-peak
    features.append(np.mean(signal))    # mean
    features.append(np.std(signal))     # standart deviation
    features.append(np.sqrt(np.mean(signal**2)))  # RMS
    rms = np.sqrt(np.mean(signal**2)) # RMS
    peak_amplitude = np.max(np.abs(signal)) #Peak
    features.append(peak_amplitude / rms if rms != 0 else 0) #cres factor
    features.append(kurtosis(signal))                        #Kurtosis
    features.append(skew(signal))                       #Skewness             
    return features

In [ ]:
X_train_features = []
X_test_features = []

for signal in X_train:
    featured_signal = extract_features(signal)
    X_train_features.append(featured_signal)
    
for signal in X_test:
    featured_signal = extract_features(signal)
    X_test_features.append(featured_signal)    


X_train_features = np.array(X_train_features) 
X_test_features =  np.array(X_test_features) 


print(X_train_features.shape)
print(X_test_features.shape)

In [ ]:
#Test different numbers of trees (from 20 to 300, in steps of 5).

estimators = list(range(20, 301, 5))
accuracies = []


for n in estimators:
    clf = RandomForestClassifier(n_estimators=n, criterion='gini', random_state=42)
    clf.fit(X_train_features, Y_train)
    Y_pred = clf.predict(X_test_features)
    acc = accuracy_score(Y_test, Y_pred)
    accuracies.append(acc)


plt.figure(figsize=(8, 5))
plt.plot(estimators, accuracies, marker='o')
plt.title('Relationship Between Accuracy and the Number of Decision Trees in the Random Forest')
plt.xlabel('Nº of Decisions Trees')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()


best_index = np.argmax(accuracies)
best_n_estimators = estimators[best_index]
best_accuracy = accuracies[best_index]

print(f"Melhor nº de árvores: {best_n_estimators}")
print(f"Accuracy correspondente: {best_accuracy:.4f}")

In [ ]:
#Find the index corresponding to 240 trees.
index_240 = estimators.index(240)

#Retrieve the corresponding accuracy.
accuracy_240 = accuracies[index_240]

print(f"Accuracy with 240 trees: {accuracy_240:.4f}")


In [ ]:
start_time = time.time()


clf = RandomForestClassifier(n_estimators=240, criterion='gini', max_depth=40, class_weight = None , random_state=42)
clf.fit(X_train_features, Y_train)


tree_depths = [estimator.tree_.max_depth for estimator in clf.estimators_]

min_depth = np.min(tree_depths)
max_depth = np.max(tree_depths)
avg_depth = np.mean(tree_depths)

print(f"Profundidade mínima: {min_depth}")
print(f"Profundidade máxima: {max_depth}")
print(f"Profundidade média: {avg_depth:.2f}")


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test_features)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
# Visualize the features importance

importances = clf.feature_importances_*100

indices = np.argsort(importances)[::-1]


feature_names = [
    "Max", "Min", "Peak", "Peak-to-pPeak", "Mean", "Standard deviation", "RMS", 
    "Crest factor", "Kurtosis", "Skewness"
]

plt.figure(figsize=(10, 6))
plt.title("Random Forest - Feature Importance")

bars = plt.bar(range(len(importances)), importances[indices], align='center')

plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
plt.ylabel("Importance (%)")
plt.ylim(0, 40)
plt.grid(axis='y', linestyle='--', alpha=0.6)

for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # posição x central da barra
        height + 0.3,                       # ligeiramente acima da barra
        f"{height:.2f}%",                   # valor com 2 casas decimais e símbolo de %
        ha='center', va='bottom', fontsize=9
    )

plt.tight_layout()
plt.show()


In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
# conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II', 'Misalig.', 'Bearings']
#conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II']
#conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=True)

plt.title('Confusion matrix')
plt.xlabel('Predicted Class')
plt.ylabel('Real Class')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve and AUC value

classes = list(labels.values())
Y_test_bin = label_binarize(Y_test, classes=classes)


Y_score = clf.predict_proba(X_test_features)


fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(classes)


for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test_bin[:, i], Y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'darkgreen', 'crimson'])

class_names = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']

for i, (color, name) in enumerate(zip(colors, class_names)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f"{name} (AUC = {roc_auc[i]:.2f})")
    

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Specificity)')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


## 02 - Random Forest with statistical features extraction and PCA

### Feature extraction

In [ ]:
def extract_features(signal):
    features = []
    features.append(np.max(signal))     # maximum
    features.append(np.min(signal))     # minimum
    features.append(np.max(np.abs(signal))) #Peak
    features.append(np.ptp(signal))         # #Peak-to-peak
    features.append(np.mean(signal))    # mean
    features.append(np.std(signal))     # standart deviation
    features.append(np.sqrt(np.mean(signal**2)))  # RMS
    rms = np.sqrt(np.mean(signal**2)) # RMS
    peak_amplitude = np.max(np.abs(signal)) #Peak
    features.append(peak_amplitude / rms if rms != 0 else 0) #cres factor
    features.append(kurtosis(signal))                        #Kurtosis
    features.append(skew(signal))                       #Skewness             
    return features

In [ ]:
X_train_features = []
X_test_features = []

for signal in X_train:
    featured_signal = extract_features(signal)
    X_train_features.append(featured_signal)
    
for signal in X_test:
    featured_signal = extract_features(signal)
    X_test_features.append(featured_signal)    
    
 
X_train_features = np.array(X_train_features) 
X_test_features =  np.array(X_test_features)   


print(X_train_features.shape)
print(X_test_features.shape)

In [ ]:
# Check whether accuracy varies with the number of principal components (PCs) applied.

results = []

for n_pca_components in range(2,11):
    
    pca = PCA(n_components = n_pca_components) 
    X_train_pca = pca.fit_transform(X_train_features)
    X_test_pca = pca.fit_transform(X_test_features)


    clf = RandomForestClassifier(n_estimators=225, criterion ='gini', class_weight = None , random_state=42)
    clf.fit(X_train_pca, Y_train)

    Y_pred = clf.predict(X_test_pca)
    
    acc = accuracy_score(Y_test, Y_pred)
    
    results.append((n_pca_components ,acc))

    print(f"n_components={n_pca_components} -> Accuracy = {acc:.4f}")


In [ ]:
# Plot showing the relationship between the number of principal components (PCs) and model accuracy.

n_principal_components, accuracies = zip(*results)

plt.figure(figsize=(8, 5))
plt.plot(n_principal_components, accuracies, marker='o')
plt.title("Random Forest Accuracy vs. Number of Principal Components")
plt.xlim(2, 10)
plt.xlabel("Number of Principal Components (PCs)")
plt.ylabel("RF model accuracy")
plt.grid(True)
plt.show()



In [ ]:
# Apply PCA

n_of_pca_components = 9   # nºo of PCA components < n_features

pca = PCA(n_components = n_of_pca_components) 
X_train_pca = pca.fit_transform(X_train_features)

X_test_pca = pca.fit_transform(X_test_features)


print("Shape after PCA:", X_train_pca.shape)

print(pca.components_.shape) 

In [ ]:
# Test different numbers of trees (20 to 300, steps of 5).

estimators = list(range(20, 301, 5))
accuracies = []

for n in estimators:
    clf = RandomForestClassifier(n_estimators=n, criterion ='gini', random_state=42)
    clf.fit(X_train_pca, Y_train)
    Y_pred = clf.predict(X_test_pca)
    acc = accuracy_score(Y_test, Y_pred)
    accuracies.append(acc)

plt.figure(figsize=(8, 5))
plt.plot(estimators, accuracies, marker='o')
plt.title('Relationship Between Accuracy and the Number of Decision Trees in the Random Forest')
plt.xlabel('Nº of Decisions Trees')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()


best_index = np.argmax(accuracies)
best_n_estimators = estimators[best_index]
best_accuracy = accuracies[best_index]

print(f"Melhor nº de árvores: {best_n_estimators}")
print(f"Accuracy correspondente: {best_accuracy:.4f}")

In [ ]:
# Find the index corresponding to 240 trees.
index_240 = estimators.index(240)

# Retrieve the corresponding accuracy.
accuracy_240 = accuracies[index_240]

print(f"Accuracy com 240 árvores: {accuracy_240:.4f}")


In [ ]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators=240, criterion='gini', max_depth=40, class_weight = None , random_state=42)
clf.fit(X_train_pca, Y_train)

tree_depths = [estimator.tree_.max_depth for estimator in clf.estimators_]

min_depth = np.min(tree_depths)
max_depth = np.max(tree_depths)
avg_depth = np.mean(tree_depths)

print(f"Profundidade mínima: {min_depth}")
print(f"Profundidade máxima: {max_depth}")
print(f"Profundidade média: {avg_depth:.2f}")


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:

Y_pred = clf.predict(X_test_pca)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

# conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
#conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Misalig.', 'Bearings']
conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II']
#conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=False)

plt.title('Confusion matrix')
plt.xlabel('Predicted Classe')
plt.ylabel('Real Classe')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve and AUC value

classes = list(labels.values())
Y_test_bin = label_binarize(Y_test, classes=classes)

Y_score = clf.predict_proba(X_test_features)

fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(classes)


for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test_bin[:, i], Y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])



plt.figure(figsize=(8, 6))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'darkgreen', 'crimson'])

# Nomes personalizados das classes
class_names = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']

for i, (color, name) in enumerate(zip(colors, class_names)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f"{name} (AUC = {roc_auc[i]:.2f})")




plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Specificity)')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


## 03 - Random Forest with FFT features extraction

### Extract frequency features (FFT)

In [ ]:
def extract_features_fft(signal):
    fft_vals = fft(signal)
    fft_magnitude = np.abs(fft_vals)[:len(signal)//2]
    
    return fft_magnitude

In [ ]:
X_train_fft = []
X_test_fft = []

for signal in X_train:
    featured_signal = extract_features_fft(signal)
    X_train_fft.append(featured_signal)
    
for signal in X_test:
    featured_signal = extract_features_fft(signal)
    X_test_fft.append(featured_signal)    
    
X_train_fft =  np.array(X_train_fft)   
X_test_fft = np.array(X_test_fft)


print(X_train_fft.shape)
print(X_test_fft.shape)

In [ ]:
# Test different numbers of trees (20 to 200, steps of 10)

estimators = list(range(20, 301, 5))
accuracies_fft = []


for n in estimators:
    clf = RandomForestClassifier(n_estimators=n, criterion ='gini', random_state=42)
    clf.fit(X_train_fft, Y_train)
    Y_pred = clf.predict(X_test_fft)
    acc = accuracy_score(Y_test, Y_pred)
    accuracies_fft.append(acc)


plt.figure(figsize=(8, 5))
plt.plot(estimators, accuracies_fft, marker='o')
plt.title('Accuracy vs Number of Decision Trees in the Random Forest')
plt.xlabel('Decisions Trees')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()


best_index = np.argmax(accuracies_fft)
best_n_estimators = estimators[best_index]
best_accuracy = accuracies_fft[best_index]

print(f"Melhor nº de árvores: {best_n_estimators}")
print(f"Accuracy correspondente: {best_accuracy:.4f}")

In [ ]:
# Find the index corresponding to 240 trees.
index_240 = estimators.index(240)

# Retrieve the corresponding accuracy.
accuracy_240 = accuracies[index_240]

print(f"Accuracy com 240 árvores: {accuracy_240:.4f}")

In [ ]:
start_time = time.time()


clf = RandomForestClassifier(n_estimators=240, criterion ='gini', max_depth=40, class_weight = None , random_state=42)
clf.fit(X_train_fft, Y_train)

tree_depths = [estimator.tree_.max_depth for estimator in clf.estimators_]

min_depth = np.min(tree_depths)
max_depth = np.max(tree_depths)
avg_depth = np.mean(tree_depths)

print(f"Profundidade mínima: {min_depth}")
print(f"Profundidade máxima: {max_depth}")
print(f"Profundidade média: {avg_depth:.2f}")


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test_fft)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
# conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Misalig.', 'Bearings']
#conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=True)

plt.title('Confusion matrix')
plt.xlabel('Predicted Class')
plt.ylabel('Real Class')
plt.tight_layout()
plt.show()

In [ ]:
# ROC Curve and AUC value


classes = list(labels.values())
Y_test_bin = label_binarize(Y_test, classes=classes)


Y_score = clf.predict_proba(X_test_fft)


fpr = dict()
tpr = dict()
roc_auc = dict()

n_classes = len(classes)


for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test_bin[:, i], Y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])



plt.figure(figsize=(8, 6))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'darkgreen', 'crimson'])

class_names = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']

for i, (color, name) in enumerate(zip(colors, class_names)):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f"{name} (AUC = {roc_auc[i]:.2f})")


plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Specificity)')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


### 04 - Random Forest with raw signal (with no feature extraction)

In [ ]:
#Signal Standardization

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Signal normalization

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Test different numbers of trees (20 to 200, steps of 10)

estimators = list(range(20, 301, 5))
accuracies = []


for n in estimators:
    clf = RandomForestClassifier(n_estimators=n, criterion ='gini', random_state=42)
    clf.fit(X_train_scaled, Y_train)
    Y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(Y_test, Y_pred)
    accuracies.append(acc)


best_index = np.argmax(accuracies)
best_n_estimators = estimators[best_index]
best_accuracy = accuracies[best_index]

print(f"Melhor nº de árvores: {best_n_estimators}")
print(f"Accuracy correspondente: {best_accuracy:.4f}")


plt.figure(figsize=(8, 5))
plt.plot(estimators, accuracies, marker='o')
plt.title('Accuracy vs Number of Decision Trees in the Random Forest')
plt.xlabel('Decisions Trees)')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(estimators, accuracies, marker='o')
plt.title('Accuracy vs Number of Decision Trees in the Random Forest')
plt.xlim([20, 300])
plt.xlabel('Decisions Trees')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()


best_index = np.argmax(accuracies)
best_n_estimators = estimators[best_index]
best_accuracy = accuracies[best_index]

print(f"Melhor nº de árvores: {best_n_estimators}")
print(f"Accuracy correspondente: {best_accuracy:.4f}")

In [ ]:
# Find the index corresponding to 240 trees.
index_240 = estimators.index(240)

# Retrieve the corresponding accuracy.
accuracy_240 = accuracies[index_240]

print(f"Accuracy com 240 árvores: {accuracy_240:.4f}")

In [ ]:
start_time = time.time()


clf = RandomForestClassifier(n_estimators=240, criterion ='gini', max_depth=40 , class_weight = None , random_state=42)
clf.fit(X_train_scaled, Y_train)


tree_depths = [estimator.tree_.max_depth for estimator in clf.estimators_]

min_depth = np.min(tree_depths)
max_depth = np.max(tree_depths)
avg_depth = np.mean(tree_depths)

print(f"Profundidade mínima: {min_depth}")
print(f"Profundidade máxima: {max_depth}")
print(f"Profundidade média: {avg_depth:.2f}")


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test)

print(confusion_matrix(Y_test_scaled, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

# conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Misalig.', 'Bearings']
conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=True)

plt.title('Confusion matrix')
plt.xlabel('Predicted Classe')
plt.ylabel('Real Classe')
plt.tight_layout()
plt.show()

### 05 - Random Forest with noisy data (Train Dataset noisy + Test Dataset not noisy)

In [ ]:
def add_white_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise


In [ ]:

X_train_noisy = np.array([add_white_noise(signal, snr_db=10) for signal in X_train]) 

# snr_db=30 → Almost noise-free.
# snr_db=10 → Moderate noise.
# snr_db=5 → High noise.

print(X_train_noisy.shape)

### Extract frequency features (FFT)

In [ ]:
def extract_features_fft(signal):
    fft_vals = fft(signal)
    fft_magnitude = np.abs(fft_vals)[:len(signal)//2]
    
    return fft_magnitude

In [ ]:
X_train_fft_noisy = []
X_test_fft = []

for signal in X_train_noisy:
    featured_signal = extract_features_fft(signal)
    X_train_fft_noisy.append(featured_signal)
    
for signal in X_test:
    featured_signal = extract_features_fft(signal)
    X_test_fft.append(featured_signal)    
    
X_train_fft_noisy =  np.array(X_train_fft_noisy)   
X_test_fft = np.array(X_test_fft)

print(X_train_fft_noisy.shape)
print(X_test_fft.shape)

In [ ]:
start_time = time.time()


clf = RandomForestClassifier(n_estimators=240, criterion ='gini', max_depth=40 , class_weight = None , random_state=42)
clf.fit(X_train_fft_noisy, Y_train)


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test_fft)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

#conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=False)

plt.title('Confusion matrix')
plt.xlabel('Predicted Classe')
plt.ylabel('Real Classe')
plt.tight_layout()
plt.show()

### 06 - Random Forest with noisy data (Train Dataset not noisy + Test Dataset noisy)

In [ ]:
def add_white_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

In [ ]:
X_test_noisy = np.array([add_white_noise(signal, snr_db=10) for signal in X_test]) 

print(X_test_noisy.shape)

### Extract frequency features (FFT)

In [ ]:
def extract_features_fft(signal):
    fft_vals = fft(signal)
    fft_magnitude = np.abs(fft_vals)[:len(signal)//2]
    
    return fft_magnitude

In [ ]:
X_train_fft = []
X_test_fft_noisy = []

for signal in X_train:
    featured_signal = extract_features_fft(signal)
    X_train_fft.append(featured_signal)
    
for signal in X_test_noisy:
    featured_signal = extract_features_fft(signal)
    X_test_fft_noisy.append(featured_signal)    
    
X_train_fft =  np.array(X_train_fft)   
X_test_fft_noisy = np.array(X_test_fft_noisy)


print(X_train_fft.shape)
print(X_test_fft_noisy.shape)

In [ ]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators=240, criterion ='gini', max_depth=40 , class_weight = None , random_state=42)
clf.fit(X_train_fft, Y_train)

end_time = time.time()
elapsed_time = end_time - start_time

print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test_fft_noisy)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

#conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=False)

plt.title('Confusion matrix')
plt.xlabel('Predicted Classe')
plt.ylabel('Real Classe')
plt.tight_layout()
plt.show()

### 07 - Random Forest with noisy data (Train Dataset noisy + Test Dataset noisy)

In [ ]:
def add_white_noise(signal, snr_db):
    signal_power = np.mean(signal**2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

In [ ]:
X_train_noisy = np.array([add_white_noise(signal, snr_db=10) for signal in X_train]) 
X_test_noisy = np.array([add_white_noise(signal, snr_db=10) for signal in X_test]) 


print(X_train_noisy.shape)
print(X_test_noisy.shape)

### Extract frequency features (FFT)

In [ ]:
def extract_features_fft(signal):
    fft_vals = fft(signal)
    fft_magnitude = np.abs(fft_vals)[:len(signal)//2]
    
    return fft_magnitude

In [ ]:
X_train_fft_noisy = []
X_test_fft_noisy = []

for signal in X_train_noisy:
    featured_signal = extract_features_fft(signal)
    X_train_fft_noisy.append(featured_signal)
    
for signal in X_test_noisy:
    featured_signal = extract_features_fft(signal)
    X_test_fft_noisy.append(featured_signal)    
    
X_train_fft_noisy =  np.array(X_train_fft_noisy)   
X_test_fft_noisy = np.array(X_test_fft_noisy)


print(X_train_fft_noisy.shape)
print(X_test_fft_noisy.shape)

In [ ]:
start_time = time.time()


clf = RandomForestClassifier(n_estimators=240, criterion ='gini', max_depth=40 , class_weight = None , random_state=42)
clf.fit(X_train_noisy, Y_train)


end_time = time.time()
elapsed_time = end_time - start_time


print("Time spent in training the model (s):", elapsed_time)

In [ ]:
Y_pred = clf.predict(X_test)

print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

In [ ]:
conf_matriz = confusion_matrix(Y_test, Y_pred)

#conf_matriz_classes = ['No unb.', 'Unb. I', 'Unb. II', 'Unb. III', 'Unb. IV']
conf_matriz_classes = ['Normal', 'Unb. I', 'Unb. II','Unb. III', 'Misalig.', 'Unb. II + Misalig.']

plt.figure(figsize=(6, 5))
sns.heatmap(conf_matriz, annot=True, fmt='d', cmap='Blues',
            xticklabels=conf_matriz_classes, yticklabels=conf_matriz_classes,cbar=False)

plt.title('Confusion matrix')
plt.xlabel('Predicted Classe')
plt.ylabel('Real Classe')
plt.tight_layout()
plt.show()